In [20]:
import numpy as np
import matplotlib.pyplot as plt

def poisson_random_process(lmbda, total_time):
    """
    Generate a Poisson random process.
    
    Parameters:
        lmbda (float): The parameter lambda of the Poisson distribution.
        total_time (float): The total time for the process.
    
    Returns:
        numpy.ndarray: An array of timestamps when events occur.
    """
    num_events = np.random.poisson(lmbda * total_time)
    event_times = np.cumsum(np.random.exponential(1 / lmbda, num_events))
    event_times = event_times[event_times < total_time]
    return event_times



In [91]:
gen_rate = 1e6 # Parameter lambda in Hz
total_time = 1e1  # Total time in sec for the process
fdl = 1e-4 # delay line time quantization in sec
tolerance = 1e-6

# s1 = poisson_random_process(gen_rate, total_time)
# s2 = poisson_random_process(gen_rate, total_time)
# # print("Event timestamps:", event_timestamps)
# np.sum(s1==s2)


In [152]:
gen_rate = 1e6 # Parameter lambda in Hz
total_time = 1e1  # Total time in sec for the process
fdl = 1e-4 # delay line time quantization in sec
tolerance = 1e-6

Nrep = 10
for _ in range(Nrep):
    s1 = poisson_random_process(gen_rate, total_time)
    s2 = poisson_random_process(gen_rate, total_time)

    num_trials = min(s1.shape[0],s2.shape[0])
    # num_trials
    all_events = np.concatenate((s1,s2))
    events_inds = all_events.argsort()
    emissions = np.zeros(events_inds.shape[0])
    emissions[np.argwhere(events_inds>s1.shape[0])] = 1
    # print(emissions[:40])
    # plt.plot(emissions,".")

    second_tick = np.argwhere(np.abs(np.diff(emissions))>0)[:,0]+1
    # print(second_tick[:10])
    # # for i in range(1,second_tick.shape[0]):
    # num_coincidences = np.sum(np.abs(np.diff(emissions))>0)

    coincident_events = []
    coincident_inds = []
    time_diff = []

    i1 = 0
    photon1 = emissions[i1]
    for _ in range(num_trials):
        # if second_tick[i]> second_tick[i-1]+1:
        #     # second_tick[i]+1:second_tick[i-1]
        #     i1 = second_tick[i-1]+1
        #     i2 = second_tick[i]
        #     print((i1,i2),emissions[[i1,i2]],all_events[[i1,i2]])
        next_event = np.argwhere(emissions[i1+1:]== 1- photon1)
        if len(next_event)>0:
            i2 = i1+ next_event[0,0] +1 
            photon2 = emissions[i2]
            # print((i1,i2),emissions[[i1,i2]],all_events[[i1,i2]])
            time_diff.append(all_events[i2]-all_events[i1])
            coincident_inds.append((i1,i2))
            coincident_events.append(emissions[[i1,i2]].tolist())
            i1 = i2 + 1
            if i1 < len(emissions):
                photon1 = emissions[i1]
            else:
                break
        else:
            break

    print(np.sum( (np.array(time_diff) % fdl)< tolerance)/len(time_diff))

0.12274959083469722
0.12386706948640483
0.06477093206951026
0.11061285500747384
0.07656967840735068
0.09584664536741214
0.11428571428571428
0.09831029185867896
0.08433734939759036
0.09361069836552749


In [89]:
print(time_diff[:10])
print(coincident_inds[:10])
print(coincident_events[:10])


[0.01795582419489879, 0.007973455803673368, 0.02586563643588935, 0.006622706650107987, 0.006762946742751186, 0.03618681398692761, 0.02378526959157007, 5.772241183898963e-05, 0.022676700654992776, 0.028509085995288563]
[(0, 2), (3, 4), (5, 6), (7, 10), (11, 12), (13, 16), (17, 20), (21, 22), (23, 28), (29, 30)]
[[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0]]


In [80]:
# gen_rate = 1e6 
# Niter = 1e4
# # s1 = np.random.poisson(gen_rate, Niter)
# # s2 = np.random.poisson(gen_rate, Niter)
# # np.argwhere(s1==s2)

912